<a href="https://colab.research.google.com/github/brytlao/Tensorflow-free-course/blob/master/LESSON_3_1_Images_of_Clothing_Additional_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise: Classifying Images of Clothing

#Install and import dependencies

In [0]:
!pip install -U tensorflow_datasets

In [0]:
# tensorflow dataset
import tensorflow as tf
import tensorflow_datasets as tfds
tf.logging.set_verbosity(tf.logging.ERROR)

# helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

# progress bar display
import tqdm
import tqdm.auto
tqdm.tqdm = tqdm.auto.tqdm

# print tf version
print(tf.__version__)

# tf issue
tf.enable_eager_execution()

#Import specific dataset

In [0]:
# import mnist dataset
dataset, metadata = tfds.load("fashion_mnist", as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [0]:
# set class labels
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [0]:
# explore imported data
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("number of training samples: {}".format(num_train_examples))
print("number of test samples: {}".format(num_test_examples))

#Preprocess the data

In [0]:
# define normalize function
def normalize(images,labels):
  images = tf.cast(images,tf.float32)
  images /= 255
  return images,labels

# normalize data
train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

In [0]:
# DONE

# extra: Don't normalize the pixel values, and see the effect that has

# just don't run the above

#Explore the data

In [0]:
# plot single sample
for image,label in train_dataset.take(1):
  break
image = image.numpy().reshape((28,28))

plt.figure()
plt.imshow(image,cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

In [0]:
# plot multiple samples
plt.figure(figsize=(10,10))
i = 0
for image,label in train_dataset.take(25):
  image = image.numpy().reshape((28,28))
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image,cmap=plt.cm.binary)
  plt.xlabel(class_names[label])
  i += 1
plt.show()

#Build the model

In [0]:
# create layers
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

# compile model
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [0]:
# DONE
# extra: Number of neurons in the Dense layer following the Flatten one.
# For example, go really low (e.g. 10) in ranges up to 512 and see how accuracy changes

# create layers
#NUM_NEURONS = 512
#model = tf.keras.Sequential([
#    tf.keras.layers.Flatten(input_shape=(28,28,1)),
#    tf.keras.layers.Dense(NUM_NEURONS, activation=tf.nn.relu),
#    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
#])

# compile model
#model.compile(optimizer="adam",
#              loss="sparse_categorical_crossentropy",
#              metrics=["accuracy"])

In [0]:
# DONE
# extra: Add additional Dense layers between the Flatten and the final Dense(10, activation=tf.nn.softmax),
# experiment with different units in these layers

# create layers
#NUM_NEURONS = 256
#model = tf.keras.Sequential([
#    tf.keras.layers.Flatten(input_shape=(28,28,1)),
#    tf.keras.layers.Dense(NUM_NEURONS, activation=tf.nn.relu),
#    tf.keras.layers.Dense(NUM_NEURONS, activation=tf.nn.relu),
#    tf.keras.layers.Dense(NUM_NEURONS, activation=tf.nn.relu),
#    tf.keras.layers.Dense(NUM_NEURONS, activation=tf.nn.relu),
#    tf.keras.layers.Dense(NUM_NEURONS, activation=tf.nn.relu),
#    tf.keras.layers.Dense(NUM_NEURONS, activation=tf.nn.relu),
#    tf.keras.layers.Dense(NUM_NEURONS, activation=tf.nn.relu),
#    tf.keras.layers.Dense(NUM_NEURONS, activation=tf.nn.relu),
#    tf.keras.layers.Dense(NUM_NEURONS, activation=tf.nn.relu),
#    tf.keras.layers.Dense(NUM_NEURONS, activation=tf.nn.relu),
#    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
#])

# compile model
#model.compile(optimizer="adam",
#              loss="sparse_categorical_crossentropy",
#              metrics=["accuracy"])

#Train the model

In [0]:
# set iteration behavior
BATCH_SIZE = 32
train_dataset = train_dataset.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

# fit model to data
NUM_EPOCH = 5
model.fit(train_dataset, epochs=NUM_EPOCH, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

In [0]:
# DONE
# extra: Set training epochs set to 1

# set iteration behavior
#BATCH_SIZE = 32
#train_dataset = train_dataset.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
#test_dataset = test_dataset.batch(BATCH_SIZE)

# fit model to data
#NUM_EPOCH = 1
#model.fit(train_dataset, epochs=NUM_EPOCH, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

#Evaluate accuracy

In [0]:
# print test accuracy
BATCH_SIZE = 32
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/BATCH_SIZE))
print("test performance: {}".format(test_accuracy))

#Make predictions

In [0]:
# test on single sample
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [0]:
# shape of prediction
print("prediction shape: {}".format(predictions.shape))

# view single sample
print("confidence values:\n{}".format(predictions[0]))

# show highest confidence value
print("highest confidence value: {}".format(np.argmax(predictions[0])))

# real label
print("real label: {}".format(test_labels[0]))

In [0]:
# define plot helper functions
def plot_image(i, predictions_array, true_labels, images):
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img[...,0], cmap=plt.cm.binary)
  
  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = "blue"
  else:
    color = "red"
    
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                       100*np.max(predictions_array),
                                       class_names[true_label]),
                                       color=color)

def plot_value_array(i,predictions_array,true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  thisplot = plt.bar(range(10),predictions_array,color="#777777")
  plt.ylim([0,1])
  
  predicted_label = np.argmax(predictions_array)
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [0]:
# plotting for one sample
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i,predictions,test_labels,test_images)
plt.subplot(1,2,2)
plot_value_array(i,predictions,test_labels)

In [0]:
# view multiple samples
num_rows = 5
num_cols = 3
num_entries = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_entries):
  plt.subplot(num_rows,2*num_cols,2*i+1)
  plot_image(i,predictions,test_labels,test_images)
  plt.subplot(num_rows,2*num_cols,2*i+2)
  plot_value_array(i,predictions,test_labels)

#Single image prediction

In [0]:
# get image from test dataset
img = test_images[0]
print(img.shape)

In [0]:
# add image to batch
img = np.array([img])
print(img.shape)

In [0]:
# predict image
predictions_single = model.predict(img)
print("predictions: \n{}".format(predictions_single))

# plot value array
plot_value_array(0,predictions_single,test_labels)
_ = plt.xticks(range(10),class_names,rotation=45)

# print class index
predict_index = np.argmax(predictions_single)
print("predicted class index: {}".format(predict_index))
print("predicted class name: {}".format(class_names[predict_index]))

# **Additional exercises**

* Set training epochs set to 1
* Number of neurons in the Dense layer following the Flatten one. For example, go really low (e.g. 10) in ranges up to 512 and see how accuracy changes
* Add additional Dense layers between the Flatten and the final Dense(10, activation=tf.nn.softmax), experiment with different units in these layers
* Don't normalize the pixel values, and see the effect that has